In [ ]:
import numpy as np

## PCA

In [ ]:
# PCA: Objetivo: Es una proyeccion ortogonal o transformacion lineal que busca maximizar la varianza de los datos proyectados
def PCA(X: np.ndarray, k):
    # Center the data
    X = X - np.mean(X, axis=0)
    
    # Calculate the covariance matrix
    cov = np.cov(X, rowvar=False)
    
    # Calculate the eigenvectors and eigenvalues
    eigvals, eigvecs = np.linalg.eig(cov)
    
    # Sort the eigenvectors by decreasing eigenvalues
    idx = np.argsort(eigvals)[::-1]
    
    # select the top k eigenvectors to form the principal matrix W
    W = eigvecs[:, idx[:k]]
    
    # Project the data onto the principal components
    Z = X @ W  # Z = XW
    return W, Z

## LDA

In [ ]:
# LDA: Maximiza la distancia entre las medias de las proyecciones de las clases y minimiza sus varianzas
def LDA(X: np.ndarray, y: np.ndarray, C, k):
    """y: vector de etiquetas de las clases, C: numero de clases"""
    m = X.shape[0], n = X.shape[1]  # X in R^{m x n}, Y in R^m

    # compute the mean vectors for each class
    mean_vectors = []
    for c in range(C):
        mean_vectors.append(np.mean(X[y == c], axis=0))
    mean_vectors = np.array(mean_vectors)

    # compute the overall mean
    overall_mean = np.mean(X, axis=0)

    # compute the between-class scatter matrix S_B
    S_B = np.zeros((X.shape[1], X.shape[1]))
    for c in range(C):
        n_c = X[y == c].shape[0]
        S_B += n_c * (mean_vectors[c] - overall_mean) @ (mean_vectors[c] - overall_mean).T

    # compute the within-class scatter matrix S_W
    S_W = np.zeros((X.shape[1], X.shape[1]))
    for c in range(C):
        S_W += np.cov(X[y == c], rowvar=False)  # covariance matrix = sum((X - mean) @ (X - mean).T) / (n - 1)

    # compute the eigenvectors and eigenvalues of S_W^-1 S_B, set \lambda as the eigenvalues and w as the eigenvectors
    eigvals, eigvecs = np.linalg.eig(np.linalg.inv(S_W) @ S_B)

    # select the top k eigenvectors corresponding to the k largest eigenvalues to form the principal matrix W
    idx = np.argsort(eigvals)[::-1]
    W = eigvecs[:, idx[:k]]

    # project the data onto the principal components
    Z = X @ W  # Z = XW

    return W, Z

## SVD

In [ ]:
# SVD: Objetivo: Descomponer una matriz en tres matrices, U, S y V, donde U y V son ortogonales y S es diagonal
def SVD(X: np.ndarray):  # X in R^{m x n}
    
# easy to use the built-in function
    U_l, Sigma_l, V_l = np.linalg.svd(X)  

# intern process algorithm
    # calculate the covariance matrix
    C = X.T @ X

    # calculate the eigenvectors and eigenvalues
    eigvals, eigvecs = np.linalg.eig(C)

    # sort the eigenvectors by decreasing eigenvalues
    idx = np.argsort(eigvals)[::-1]

    # form the matrix V using the top n eigenvectors
    V = eigvecs[:, idx]

    # compute the singular values \sigma as the square root of the eigenvalues of C
    sigma = np.sqrt(eigvals[idx])
    # form the matrix sigma with the singular values on the diagonal
    Sigma = np.diag(sigma)

    # compute the matrix U using the formula U = X @ V @ Sigma^-1
    U = X @ V @ np.linalg.inv(Sigma)

    return U, Sigma, V

## NMMF

In [ ]:
# NMMF: Objetivo: Descomponer una matriz en dos matrices no negativas, similar a la SVD
def NMMF(X: np.ndarray, rank, max_iter=100, toler=0.001): # X in R^{m x n}
    # NOTE: we use multiplicative update rules for non-negative matrix factorization

    # init W and H
    W = np.random.rand(X.shape[0], rank)
    H = np.random.rand(rank, X.shape[1])

    for _ in range(max_iter):
        # update W
        W = W * ((X @ H.T) / (W @ H @ H.T))

        # update H
        H = H * ((W.T @ X) / (W.T @ W @ H))

        # compute the error
        error = np.linalg.norm(X - W @ H)
        if error < toler:
            break
        
    return W, H

## Random Projections

In [ ]:
# Random Projection: Objetivo: Proyectar los datos en un espacio de menor dimension, donde se preserva la distancia entre los puntos en el espacio original
def RandomProjection(X: np.ndarray, target_dim):
    # generate a random projection matrix
    # here we use Gaussian random matrix ( \mathcal{N}(0, 1/target_dim) )
    # it means, R_ij ~ \mathcal{N}(0, 1/target_dim)
    R = np.random.randn(X.shape[1], target_dim) / np.sqrt(target_dim)
    
    # project the data matrix X onto the lower-dimensional space using R
    Z = X @ R  # Z = XR

    # optionally, normalize the projected data Z to preserve the Euclidean distance
    Z = Z / np.sqrt(target_dim)

    return Z

## t-SNE

In [ ]:
# t-SNE: Utiliza la distribucion de probabilidad de la vecindad de los puntos para proyectar los datos en un espacio de menor dimension
def tSNE(X: np.ndarray, target_dim, perplexity=30, max_iter=1000, learning_rate=0.1, toler=0.001):
    # let perplexity  as \sigma_i^2 for each point i
    # for each pair of points i and j, compute the conditional probability p_{j|i}
    # that x_j is the neighbor of x_i, using the Gaussian kernel
    m = X.shape[0], n = X.shape[1]  # X in R^{m x n}

    def compute_pairwise_probabilities(X, perplexity):
        n = X.shape[0]
        P = np.zeros((n, n))
        for i in range(n):
            dists = np.linalg.norm(X - X[i], axis=1)
            p = np.exp(-dists ** 2 / (2 * perplexity ** 2))
            p[i] = 0  # set the self-probability to 0
            p = p / p.sum()  # normalize the probabilities
            P[i] = p
        return P
    
    # compute the pairwise probabilities
    P = compute_pairwise_probabilities(X, perplexity)

    # symmetrize the probabilities to get the joint probabilities
    P = (P + P.T) / (2 * m)

    # initialize the low-dimensional representation Y \in R^{m x target_dim} using Gaussian random noise
    Y = np.random.randn(m, target_dim)

    # compute the pairwise similarities in the low-dimensional space. omputing for each pair q_{ij} using the Student's t-distribution with one degree of freedom
    def compute_pairwise_similarities(Y):
        n = Y.shape[0]
        Q = np.zeros((n, n))
        for i in range(n):
            dists = np.linalg.norm(Y - Y[i], axis=1)
            q = 1 / (1 + dists ** 2)
            q[i] = 0  # set the self-similarity to 0
            q = q / q.sum()  # normalize the similarities
            Q[i] = q
        return Q
    
    # compute the pairwise similarities in the low-dimensional space
    Q = compute_pairwise_similarities(Y)
    # Minimize the Kullback-Leibler divergence between P and Q
    C = np.sum(P * np.log(P / Q))

    # perfom the gradient descent to minimize the KL divergence
    
    for _ in range(max_iter):
        # compute the gradient of the KL divergence
        dC = np.zeros((m, target_dim))
        for i in range(m):
            dC[i] = 4 * np.sum((P[i] - Q[i])[:, None] * (Y[i] - Y), axis=0)

        # update the low-dimensional representation Y
        Y = Y - learning_rate * dC

        # compute the pairwise similarities in the low-dimensional space
        Q = compute_pairwise_similarities(Y)
        # compute the KL divergence
        C_new = np.sum(P * np.log(P / Q))

        # check the convergence
        if np.abs(C - C_new) < toler:
            break
        C = C_new

    return Y

## UMAP

In [ ]:
# UMAP: Es similar al t-SNE, utiliza la distribucion de probabilidad de la vecindad de los puntos para proyectar los datos en un espacio de menor dimension
def UMAP(X: np.ndarray, target_dim, n_neighbors=15, min_dist=0.1, n_epochs=100, learning_rate=0.1, toler=0.001):
    # compute the fuzzy simplicial set
    # compute the k-nearest neighbors graph
    k = n_neighbors
    m = X.shape[0], n = X.shape[1]  # X in R^{m x n}

    def compute_knn_graph(X, k):
        n = X.shape[0]
        G = np.zeros((n, n))
        for i in range(n):
            dists = np.linalg.norm(X - X[i], axis=1)
            idx = np.argsort(dists)[1:k+1]  # exclude the self-neighbor
            G[i, idx] = 1
        return G
    
    # compute the k-nearest neighbors graph
    G = compute_knn_graph(X, k)

    # compute the membership strength using the Gaussian kernel
    def compute_membership_strength(X, G, min_dist):
        n = X.shape[0]
        A = np.zeros((n, n))
        for i in range(n):
            idx = np.where(G[i] == 1)[0]
            dists = np.linalg.norm(X[idx] - X[i], axis=1)
            a = np.exp(-dists / min_dist)
            A[i, idx] = a
        return A
    
    # compute the membership strength
    A = compute_membership_strength(X, G, min_dist)

    # symmetrize the membership strength, wuer w^sym_{ij} = w_{ij} + w_{ji} - w_{ij} w_{ji}
    A = A + A.T - A * A.T

    # init the matrix Y
    Y = np.random.randn(m, target_dim)

    # Optimize the low-dimensional representation Y
    for _ in range(n_epochs):
        # compute the fuzzy simplicial set
        # q_{ij} = (1 + ||y_i - y_j||^2)^-1
        Q = np.zeros((m, m))
        for i in range(m):
            dists = np.linalg.norm(Y - Y[i], axis=1)
            q = 1 / (1 + dists ** 2)
            Q[i] = q
        Q = Q / Q.sum()

        # minimize the cross entropy between A and Q
        C = np.sum(A * np.log(A / Q))

        # use gradient descent to minimize the cross entropy
        dC = np.zeros((m, target_dim))
        for i in range(m):
            dC[i] = 2 * np.sum(A[i][:, None] * (Y[i] - Y), axis=0)

        # update the low-dimensional representation Y
        Y = Y - learning_rate * dC
        if np.linalg.norm(dC) < toler:
            break

    return Y